# Setup

In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

path['project'] : D:\projects\skin-lesion-classification
path['images'] : D:\projects\skin-lesion-classification\images
path['models'] : D:\projects\skin-lesion-classification\models
path['expository'] : D:\projects\skin-lesion-classification\expository
path['literature'] : D:\projects\skin-lesion-classification\literature
path['notebooks'] : D:\projects\skin-lesion-classification\notebooks
path['presentation'] : D:\projects\skin-lesion-classification\presentation
path['scripts'] : D:\projects\skin-lesion-classification\scripts
path['streamlit'] : D:\projects\skin-lesion-classification\streamlit


<a id='model_setup'></a>
# Model setup

In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

restrict_to: Union[None, dict] = {'dx' : ['mel', 'nv']}

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = { 'mel' : ['mel'], 'nv' : ['nv']}

train_one_img_per_lesion: Union[None, bool] = True

In [3]:
# Create an instance of the process class with attribute values as above.
melnv_t1 = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  restrict_to=restrict_to,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,)

- Loaded file 'D:\projects\skin-lesion-classification\images\metadata.csv'.
- Removing all records unless:
  dx in ['mel', 'nv']
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'mel': 0, 'nv': 1}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Training set (not balanced, one image per lesion): self.df_train
- Validation set (not expanded, one image per lesion): self.df_val1
- Validation set (not expanded, use all images of each lesion): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, self._df_val_a_code_test


In [4]:
import torchvision.transforms as transforms
transform = transforms.Compose([
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [5]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = melnv_t1        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "melnv_base"   # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
base_learning_rate: Union[None, float] = 0.01
unfreeze_all: Union[None, bool] = True

In [6]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_melnv_t1 = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,
                    base_learning_rate=base_learning_rate,
                    unfreeze_all=unfreeze_all,)

New files will be created. 
Base filename: rn18_t1_ufall_10e_melnv_base_01
Attributes saved to file: D:\projects\skin-lesion-classification\models\rn18_t1_ufall_10e_melnv_base_01_attributes.json


In [7]:
# import time

# tic = time.time()
# rn18_melnv_t1.train()
# toc = time.time()
# print(f"Elapsed time: {toc - tic}")

In [8]:
# from utils import print_header
# import torch
# import torch.nn as nn

# instance = melnv_t1

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [10]:
# from utils import print_header
# from multiclass_models import get_probabilities

# instance = rn18_melnv_t1

# tic = time.time()

# instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
#                                                    data_dir=instance.data_dir,
#                                                    model_dir=instance.model_dir,
#                                                    model=instance.model,
#                                                    filename=instance._filename,
#                                                    label_codes=instance.label_codes,
#                                                    transform=instance.transform,
#                                                    batch_size=instance.batch_size,
#                                                    Print=False,
#                                                    save_as=instance._filename + "_val1",)

# toc = time.time()

# print(f"Elapsed time: {toc - tic}")

# instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
#                                                     data_dir=instance.data_dir,
#                                                     model_dir=instance.model_dir,
#                                                     model=instance.model,
#                                                     filename=instance._filename,
#                                                     label_codes=instance.label_codes,
#                                                     transform=instance.transform,
#                                                     batch_size=instance.batch_size,
#                                                     Print=False,
#                                                     save_as=instance._filename + "_val_a",)
# tic = time.time()
# print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_ufall_10e_melnv_base_00_val1_probabilities.csv
Elapsed time: 18.10058069229126
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_ufall_10e_melnv_base_00_val_a_probabilities.csv
Elapsed time: 23.145163536071777


In [7]:
instance = rn18_melnv_t1

file_path1 = instance.model_dir.joinpath("rn18_t1_ufall_10e_melnv_base_00_val1_probabilities.csv")
file_path_a = instance.model_dir.joinpath("rn18_t1_ufall_10e_melnv_base_00_val_a_probabilities.csv")

instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [8]:
from utils import print_header

instance = rn18_melnv_t1

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_mel,prob_nv
0,HAM_0001751,ISIC_0024698,nv,0.218357,0.781643
1,HAM_0005678,ISIC_0031023,mel,0.411312,0.588688
2,HAM_0005191,ISIC_0031177,mel,0.370893,0.629107
3,HAM_0004476,ISIC_0030417,mel,0.176949,0.823051
4,HAM_0000876,ISIC_0032396,mel,0.289566,0.710434



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_mel,prob_nv
0,HAM_0001751,ISIC_0024698,nv,0.218357,0.781643
1,HAM_0005678,ISIC_0031023,mel,0.411312,0.588688
2,HAM_0005678,ISIC_0028086,mel,0.435894,0.564106
3,HAM_0005191,ISIC_0031177,mel,0.370893,0.629107
4,HAM_0004476,ISIC_0030417,mel,0.176949,0.823051


In [9]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_melnv_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val1)

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a)


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_mel,prob_nv,pred,pred_final
0,HAM_0001751,1,ISIC_0024698,nv,1,consensus,70.0,male,face,v1,0.218357,0.781643,1,1
1,HAM_0005678,2,ISIC_0031023,mel,0,histo,60.0,male,chest,v1,0.411312,0.588688,1,1
2,HAM_0005191,1,ISIC_0031177,mel,0,histo,40.0,female,back,v1,0.370893,0.629107,1,1
3,HAM_0004476,1,ISIC_0030417,mel,0,histo,70.0,male,face,v1,0.176949,0.823051,1,1
4,HAM_0000876,2,ISIC_0032396,mel,0,histo,55.0,male,abdomen,v1,0.289566,0.710434,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,HAM_0004857,2,ISIC_0031222,nv,1,consensus,70.0,male,lower extremity,v1,0.155081,0.844919,1,1
1501,HAM_0003384,2,ISIC_0028438,nv,1,consensus,55.0,female,chest,v1,0.055855,0.944145,1,1
1502,HAM_0001410,2,ISIC_0024830,nv,1,consensus,15.0,female,chest,v1,0.162379,0.837621,1,1
1503,HAM_0003322,2,ISIC_0031649,nv,1,consensus,50.0,female,face,v1,0.156353,0.843647,1,1



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_mel,prob_nv,pred,pred_final
0,HAM_0001751,1,ISIC_0024698,nv,1,consensus,70.0,male,face,v1,0.218357,0.781643,1,1
1,HAM_0005678,2,ISIC_0031023,mel,0,histo,60.0,male,chest,v1,0.411312,0.588688,1,1
2,HAM_0005678,2,ISIC_0028086,mel,0,histo,60.0,male,chest,va,0.435894,0.564106,1,1
3,HAM_0005191,1,ISIC_0031177,mel,0,histo,40.0,female,back,v1,0.370893,0.629107,1,1
4,HAM_0004476,1,ISIC_0030417,mel,0,histo,70.0,male,face,v1,0.176949,0.823051,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,HAM_0001410,2,ISIC_0024830,nv,1,consensus,15.0,female,chest,v1,0.162371,0.837630,1,1
1963,HAM_0004857,2,ISIC_0029038,nv,1,consensus,70.0,male,lower extremity,va,0.189378,0.810622,1,1
1964,HAM_0004857,2,ISIC_0031222,nv,1,consensus,70.0,male,lower extremity,v1,0.155075,0.844925,1,1
1965,HAM_0003384,2,ISIC_0031299,nv,1,consensus,55.0,female,chest,va,0.045434,0.954566,1,1


In [11]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_melnv_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,mel,nv,All,recall
actual,,,,
mel,0,154,154,0.0
nv,0,"1,351","1,351",1.0
All,0,"1,505","1,505",_
precision,_,0.897674,_,_



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,mel,nv,All,recall
actual,,,,
mel,0,154,154,0.0
nv,0,"1,351","1,351",1.0
All,0,"1,505","1,505",_
precision,_,0.897674,_,_


In [12]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_melnv_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.897674,0.5,0.897674,0.5,0.458215,0.473039,0.488855,0.0,NaN,NaN,NaN



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.897674,0.5,0.897674,0.5,0.458215,0.473039,0.488855,0.0,NaN,NaN,NaN


In [10]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_melnv_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel'], 'min' : ['nv']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.3)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = OrderedDict([('nv',0.7)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = OrderedDict([('mel',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val1)

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a)


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_mel,prob_nv,pred,pred_final
0,HAM_0001751,1,ISIC_0024698,nv,1,consensus,70.0,male,face,v1,0.218357,0.781643,1,1
1,HAM_0005678,2,ISIC_0031023,mel,0,histo,60.0,male,chest,v1,0.411312,0.588688,0,0
2,HAM_0005191,1,ISIC_0031177,mel,0,histo,40.0,female,back,v1,0.370893,0.629107,0,0
3,HAM_0004476,1,ISIC_0030417,mel,0,histo,70.0,male,face,v1,0.176949,0.823051,1,1
4,HAM_0000876,2,ISIC_0032396,mel,0,histo,55.0,male,abdomen,v1,0.289566,0.710434,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,HAM_0004857,2,ISIC_0031222,nv,1,consensus,70.0,male,lower extremity,v1,0.155081,0.844919,1,1
1501,HAM_0003384,2,ISIC_0028438,nv,1,consensus,55.0,female,chest,v1,0.055855,0.944145,1,1
1502,HAM_0001410,2,ISIC_0024830,nv,1,consensus,15.0,female,chest,v1,0.162379,0.837621,1,1
1503,HAM_0003322,2,ISIC_0031649,nv,1,consensus,50.0,female,face,v1,0.156353,0.843647,1,1



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_mel,prob_nv,pred,pred_final
0,HAM_0001751,1,ISIC_0024698,nv,1,consensus,70.0,male,face,v1,0.218357,0.781643,1,1
1,HAM_0005678,2,ISIC_0031023,mel,0,histo,60.0,male,chest,v1,0.411312,0.588688,0,0
2,HAM_0005678,2,ISIC_0028086,mel,0,histo,60.0,male,chest,va,0.435894,0.564106,0,0
3,HAM_0005191,1,ISIC_0031177,mel,0,histo,40.0,female,back,v1,0.370893,0.629107,0,0
4,HAM_0004476,1,ISIC_0030417,mel,0,histo,70.0,male,face,v1,0.176949,0.823051,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,HAM_0001410,2,ISIC_0024830,nv,1,consensus,15.0,female,chest,v1,0.162371,0.837630,1,1
1963,HAM_0004857,2,ISIC_0029038,nv,1,consensus,70.0,male,lower extremity,va,0.189378,0.810622,1,1
1964,HAM_0004857,2,ISIC_0031222,nv,1,consensus,70.0,male,lower extremity,v1,0.155075,0.844925,1,1
1965,HAM_0003384,2,ISIC_0031299,nv,1,consensus,55.0,female,chest,va,0.045434,0.954566,1,1


In [21]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_melnv_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,mel,nv,All,recall
actual,,,,
mel,107,47,154,0.694805
nv,155,"1,196","1,351",0.88527
All,262,"1,243","1,505",_
precision,0.408397,0.962188,_,0.638937



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,mel,nv,All,recall
actual,,,,
mel,115,39,154,0.746753
nv,184,"1,167","1,351",0.863805
All,299,"1,206","1,505",_
precision,0.384615,0.967662,_,0.654345


In [22]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_melnv_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=agg_probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=agg_probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.865781,0.790038,0.685293,0.790038,0.695423,0.718276,0.754497,0.463646,NaN,NaN,NaN



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.851827,0.805279,0.676139,0.805279,0.685433,0.710257,0.755584,0.463773,NaN,NaN,NaN
